originally taken from https://sookocheff.com/post/vim/neovim-java-ide/?fbclid=IwAR3rEh6AFYFPcozfS4v2sjOXN7f2VJPn8gWOO8Rr9DMKd0-EhcgoG-92L44

# Installing Language Server
## Language Server
### on mac
```
brew install jdtls
```

### on ubuntu

In [ ]:
%%bash
mkdir  ~/.java/installs/java
git clone git@github.com:eclipse-jdtls/eclipse.jdt.ls.git  ~/.java/installs/java

cd  ~/.java/installs/java/eclipse.jdt.ls
./mvnw clean verify

now your LSP server is build in the `/.java/installs/java/eclipse.jdt.ls/org.eclipse.jdt.ls.product/target/repository` folder.

## Debug Server

In [ ]:
%%bash
git clone git@github.com:microsoft/java-debug.git ~/.java/installs/java

cd ~/.java/installs/java/com.microsoft.java.debug.plugin
./mvnw clean install

## Debuging Adaptor Protocol

In [ ]:
%%bash
git clone git@github.com:microsoft/vscode-java-test.git ~/.java/installs/java

cd ~/.java/installs/java/vscode-java-test
npm install
npm run build-plugin

# Configuring Neo Vim

add to `plugins-setup.lua`

```
  use("hrsh7th/cmp-vsnip")
  use("hrsh7th/vim-vsnip")
  
    -- java integration
  use("mfussenegger/nvim-jdtls") -- java language server  
  use("mfussenegger/nvim-dap") -- debugger

  use("folke/which-key.nvim")
```

add to `keymaps.lua`
```
-- nvim- dap
keymap.set("n", "<leader>bb", "<cmd>lua require'dap'.toggle_breakpoint()<cr>")
keymap.set("n", "<leader>bc", "<cmd>lua require'dap'.set_breakpoint(vim.fn.input('Breakpoint condition: '))<cr>")
keymap.set("n", "<leader>bl", "<cmd>lua require'dap'.set_breakpoint(nil, nil, vim.fn.input('Log point message: '))<cr>")
keymap.set("n", "<leader>br", "<cmd>lua require'dap'.clear_breakpoints()<cr>")
keymap.set("n", "<leader>ba", '<cmd>Telescope dap list_breakpoints<cr>')
require("which-key").register({
  v = {
   name = "test",
  },
}, {prefix = "<leader>"})

keymap.set("n", "<leader>dc", "<cmd>lua require'dap'.continue()<cr>")
keymap.set("n", "<leader>dj", "<cmd>lua require'dap'.step_over()<cr>")
keymap.set("n", "<leader>dk", "<cmd>lua require'dap'.step_into()<cr>")
keymap.set("n", "<leader>do", "<cmd>lua require'dap'.step_out()<cr>")
keymap.set("n", "<leader>dd", "<cmd>lua require'dap'.disconnect()<cr>")
keymap.set("n", "<leader>dt", "<cmd>lua require'dap'.terminate()<cr>")
keymap.set("n", "<leader>dr", "<cmd>lua require'dap'.repl.toggle()<cr>")
keymap.set("n", "<leader>dl", "<cmd>lua require'dap'.run_last()<cr>")
keymap.set("n", "<leader>di", function() require"dap.ui.widgets".hover() end)
keymap.set("n", "<leader>d?", function() local widgets=require"dap.ui.widgets";widgets.centered_float(widgets.scopes) end)
keymap.set("n", "<leader>df", '<cmd>Telescope dap frames<cr>')
keymap.set("n", "<leader>dh", '<cmd>Telescope dap commands<cr>')
require("which-key").register({
 d = {
   name = "debug",
 },
}, {prefix = "<leader>"})
```

### create the file ~/.config/nvim/lua/usename/plugins/lsp/java.lua

In [ ]:
%%bash
touch ~/.config/nvim/lua/$USER/plugins/lsp/java.lua

~/.config/nvim/lua/usename/plugins/lsp/java.lua

```
local home = os.getenv('HOME')
local jdtls = require('jdtls')

-- File types that signify a Java project's root directory. This will be
-- used by eclipse to determine what constitutes a workspace
local root_markers = {'gradlew', 'mvnw', '.git'}
local root_dir = require('jdtls.setup').find_root(root_markers)

-- eclipse.jdt.ls stores project specific data within a folder. If you are working
-- with multiple different projects, each project must use a dedicated data directory.
-- This variable is used to configure elipse to use the directory name of the
-- current project found using the root_marker as the folder for project specific data.
local workspace_folder = home .. "/.local/share/eclipse/" .. vim.fn.fnamemodify(root_dir, ":p:h:t")

-- local remap = require("me.util").remap

-- Helper function for creating keymaps
function nnoremap(rhs, lhs, bufopts, desc)
  bufopts.desc = desc
  vim.keymap.set("n", rhs, lhs, bufopts)
end

-- The on_attach function is used to set keymaps after the language server
-- attaches to the current buffer
local on_attach = function(client, bufnr)
  -- With hotcodereplace = 'auto' the debug adapter will try to apply code changes
  -- you make during a debug session immediately
  jdtls.setup_dap({hotcoderplace = 'auto'})

  -- Regular Neovim LSP client keymappings
  local bufopts = {noremap = true, silent = true, buffer=bufnr}
  nnoremap('gD', vim.lsp.buf.declaration, bufopts, "Go to declaration")
  nnoremap('gd', vim.lsp.buf.definition, bufopts, "Go to definition")
  nnoremap('gi', vim.lsp.buf.implementation, bufopts, "Go to implementation")
  nnoremap('K', vim.lsp.buf.hover, bufopts, "Hover text")
  nnoremap('<C-k>', vim.lsp.buf.signature_help, bufopts, "Show signature")
  nnoremap('<space>wa', vim.lsp.buf.add_workspace_folder, bufopts, "Add workspace folder")
  nnoremap('<space>wr', vim.lsp.buf.remove_workspace_folder, bufopts, "Remove workspace folder")
  nnoremap('<space>wl', function()
    print(vim.inspect(vim.lsp.buf.list_workspace_folders()))
  end, bufopts, "Remove workspace folder")
  nnoremap('<space>D', vim.lsp.buf.type_definition, bufopts, "Go to type definition")
  nnoremap('<space>rn', vim.lsp.buf.rename, bufopts, "Rename")
  nnoremap('<space>ca', vim.lsp.buf.code_action, bufopts, "Code actions")
  vim.keymap.set('v', "<space>ca", "<ESC><CMD>lua vim.lsp.buf.range_code_action()<CR>",
    { noremap=true, silent=true, buffer=bufnr, desc = "Code actions" })
  nnoremap('<space>F', function() vim.lsp.buf.format { async = true } end, bufopts, "Format file")

  -- Java extensions provided by jdtls
  nnoremap("<C-o>", jdtls.organize_imports, bufopts, "Organize imports")
  nnoremap("<leader>tc", jdtls.test_class, bufopts, "Test class (DAP)")
  nnoremap("<leader>tm", jdtls.test_nearest_method, bufopts, "Test method (DAP)")
  nnoremap("<leader>ve", jdtls.extract_variable, bufopts, "Extract variable")
  nnoremap("<leader>ce", jdtls.extract_constant, bufopts, "Extract constant")
  vim.keymap.set('v', "<space>em", [[<ESC><CMD>lua require('jdtls').extract_method(true)<CR>]],
    { noremap=true, silent=true, buffer=bufnr, desc = "Extract method" })
end

local bundles = {
  vim.fn.glob(home .. "/.java/installs/java/com.microsoft.java.debug.plugin/target/com.microsoft.java.debug.plugin-<jar version>.jar")
}
vim.list_extend(bundles, vim.split(vim.fn.glob(home .. '/.java/installs/java/vscode-java-test/server/*jar'), "\n"))

local capabilities = vim.lsp.protocol.make_client_capabilities()
capabilities = require('cmp_nvim_lsp').default_capabilities(capabilities)

local config = {
  flags = {
    debounce_text_changes = 80,
  },
  capabilities = capabilities,
  on_attach = on_attach,  -- We pass our on_attach keybindings to the configuration map
  init_options = {
    bundles = bundles
  },
  root_dir = root_dir, -- Set the root directory to our found root_marker
  -- Here you can configure eclipse.jdt.ls specific settings
  -- These are defined by the eclipse.jdt.ls project and will be passed to eclipse when starting.
  -- See https://github.com/eclipse/eclipse.jdt.ls/wiki/Running-the-JAVA-LS-server-from-the-command-line#initialize-request
  -- for a list of options
  settings = {
    java = {
      format = {
        settings = {
          -- Use Google Java style guidelines for formatting
          -- To use, make sure to download the file from https://github.com/google/styleguide/blob/gh-pages/eclipse-java-google-style.xml
          -- and place it in the ~/.local/share/eclipse directory
          url = "/.local/share/eclipse/eclipse-java-google-style.xml",
          profile = "GoogleStyle",
        },
      },
      signatureHelp = { enabled = true },
      contentProvider = { preferred = 'fernflower' },  -- Use fernflower to decompile library code
      -- Specify any completion options
      completion = {
        favoriteStaticMembers = {
          "org.hamcrest.MatcherAssert.assertThat",
          "org.hamcrest.Matchers.*",
          "org.hamcrest.CoreMatchers.*",
          "org.junit.jupiter.api.Assertions.*",
          "java.util.Objects.requireNonNull",
          "java.util.Objects.requireNonNullElse",
          "org.mockito.Mockito.*"
        },
        filteredTypes = {
          "com.sun.*",
          "io.micrometer.shaded.*",
          "java.awt.*",
          "jdk.*", "sun.*",
        },
      },
      -- Specify any options for organizing imports
      sources = {
        organizeImports = {
          starThreshold = 9999;
          staticStarThreshold = 9999;
        },
      },
      -- How code generation should act
      codeGeneration = {
        toString = {
          template = "${object.className}{${member.name()}=${member.value}, ${otherMembers}}"
        },
        hashCodeEquals = {
          useJava7Objects = true,
        },
        useBlocks = true,
      },
      -- If you are developing in projects with different Java versions, you need
      -- to tell eclipse.jdt.ls to use the location of the JDK for your Java version
      -- See https://github.com/eclipse/eclipse.jdt.ls/wiki/Running-the-JAVA-LS-server-from-the-command-line#initialize-request
      -- And search for `interface RuntimeOption`
      -- The `name` is NOT arbitrary, but must match one of the elements from `enum ExecutionEnvironment` in the link above
      configuration = {
        runtimes = {
          {
            name = "JavaSE-17",
            path = "<java 17 path>",
          },
          {
            name = "JavaSE-11",
            path ="<java 11 path>",
          },
          {
            name = "JavaSE-1.8",
            path = ""<java home path>",
          },
        }
      }
    }
  },
  -- cmd is the command that starts the language server. Whatever is placed
  -- here is what is passed to the command line to execute jdtls.
  -- Note that eclipse.jdt.ls must be started with a Java version of 17 or higher
  -- See: https://github.com/eclipse/eclipse.jdt.ls#running-from-thecommand-line
  -- for the full list of options
  cmd = {
    home .. "<java 17 path>/bin/java",
    '-Declipse.application=org.eclipse.jdt.ls.core.id1',
    '-Dosgi.bundles.defaultStartLevel=4',
    '-Declipse.product=org.eclipse.jdt.ls.core.product',
    '-Dlog.protocol=true',
    '-Dlog.level=ALL',
    '-Xmx4g',
    '--add-modules=ALL-SYSTEM',
    '--add-opens',
    'java.base/java.util=ALL-UNNAMED',
    '--add-opens',
    'java.base/java.lang=ALL-UNNAMED',
    -- If you use lombok, download the lombok jar and place it in ~/.local/share/eclipse/
    '-javaagent:' .. home .. '/.local/share/eclipse/lombok.jar',

    -- The jar file is located where jdtls was installed. This will need to be updated
    -- to the location where you installed jdtls
    '-jar',
    vim.fn.glob(home .. '/.java/installs/java/eclipse.jdt.ls/org.eclipse.jdt.ls.product/target/repository/plugins/org.eclipse.equinox.launcher_<launcher version>.jar'),

    -- The configuration for jdtls is also placed where jdtls was installed. This will
    -- need to be updated depending on your environment
    '-configuration',
    home .. '/.java/installs/java/eclipse.jdt.ls/org.eclipse.jdt.ls.product/target/repository/<config_linux if using linux, config_mac if using mac>',

    -- Use the workspace_folder defined above to store data for this project
    '-data',
    workspace_folder,
  },
}

local M = {}
function M.make_jdtls_config()
  return config
end
return M
```

### Create file `~/.config/nvim/lua/username/plugins/nvm-dap.lua`

In [ ]:
%%bash
touch ~/.config/nvim/lua/$USER/plugins/nvm-dap.lua

~/.config/nvim/lua/username/plugins/nvm-dap.lua
```
require('dap.ext.vscode').load_launchjs()

local dap = require('dap')
dap.defaults.fallback.terminal_win_cmd = 'tabnew'
```

### add to init.lua
```
require(string.format("%s.plugins.nvm-dap", username))
```

### create ftplugin/java.lua

In [ ]:
%%bash
mkdir  ~/.config/nvim/ftplugin
touch ~/.config/nvim/ftplugin/java.lua

ftplugin/java.lua
```
local username = os.getenv("USER") or os.getenv("USERNAME")
local config = require(string.format("%s.plugins.lsp.java", username)).make_jdtls_config()
require("jdtls").start_or_attach(config)
```